In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Input, Reshape, Dense, Layer, Concatenate, Dropout
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv(r"F:\PROJECTS\AI PROJECTS\HANDRECOGPROJECT\FINAL_GITHUB_DEPOSIT\hand_landmarks_with_invariance.csv")

In [3]:
df_new = df.drop('image',axis=1)

In [4]:
X = df_new.drop(columns = ['gesture'])
y = df_new['gesture']

In [5]:
label_to_num = {label: i for i, label in enumerate(y.unique())}
y = y.replace(label_to_num).astype('int32')

C:\Users\vansh\AppData\Local\Temp\ipykernel_15684\3745509944.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace(label_to_num).astype('int32')


In [6]:
class SpatialTransformer(Layer):
    def __init__(self, output_dim=(42,), **kwargs):  # Critical fix here
        super().__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        self.localization = Sequential([
            Dense(32, activation="relu"),
            Dense(6, 
                kernel_initializer='zeros',
                bias_initializer=tf.constant_initializer([1,0,0,0,1,0]))
        ])
        super().build(input_shape)
    
    def call(self, inputs):
        x = Reshape((21, 2))(inputs)
        theta = Reshape((2, 3))(self.localization(inputs))
        
        # Correct affine transformation (21,2) → (21,3) → keep (x,y)
        transformed = tf.einsum('bij,bjk->bik', x, theta)
        return Reshape(self.output_dim)(transformed[:, :, :2])  # Maintain 42 features


In [7]:
def build_self_aware_rotational_model(input_shape=(42,), num_classes=8):
    inputs = Input(shape=input_shape)
    
    # Initialize with output_dim=42
    stn_features = SpatialTransformer(output_dim=(42,))(inputs)  # Fixed dimension
    
    # Rest remains unchanged
    rotation_features = Dense(32, activation="relu")(inputs)
    combined = Concatenate()([stn_features, rotation_features])
    x = Dense(128, activation='tanh', kernel_regularizer=l1_l2())(combined)
    x = Dropout(0.4)(x)
    x = Dense(32, activation='tanh', kernel_regularizer=l1_l2())(x)
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs=inputs, outputs=outputs)


In [8]:
model = build_self_aware_rotational_model()

In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=0.1,random_state=0)

In [11]:
train_X.dtypes

r1         float64
theta1     float64
r2         float64
theta2     float64
r3         float64
theta3     float64
r4         float64
theta4     float64
r5         float64
theta5     float64
r6         float64
theta6     float64
r7         float64
theta7     float64
r8         float64
theta8     float64
r9         float64
theta9     float64
r10        float64
theta10    float64
r11        float64
theta11    float64
r12        float64
theta12    float64
r13        float64
theta13    float64
r14        float64
theta14    float64
r15        float64
theta15    float64
r16        float64
theta16    float64
r17        float64
theta17    float64
r18        float64
theta18    float64
r19        float64
theta19    float64
r20        float64
theta20    float64
r21        float64
theta21    float64
dtype: object

In [12]:
history = model.fit(train_X,train_y,epochs=30,batch_size=16,validation_split=0.15)

Epoch 1/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5227 - loss: 1.2999 - val_accuracy: 0.8440 - val_loss: 0.2909
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9410 - loss: 0.2768 - val_accuracy: 0.9929 - val_loss: 0.0685
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9895 - loss: 0.0886 - val_accuracy: 0.9953 - val_loss: 0.0375
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9950 - loss: 0.0523 - val_accuracy: 0.9976 - val_loss: 0.0237
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9979 - loss: 0.0289 - val_accuracy: 0.9953 - val_loss: 0.0157
Epoch 6/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9967 - loss: 0.0221 - val_accuracy: 0.9953 - val_loss: 0.0213
Epoch 7/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9965 - loss: 0.0241 - val_accuracy: 0.9976 - val_loss: 0.0124
Epoch 8/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9987 - loss: 0.0161 - val_accuracy: 0.

In [14]:
loss,accuracy = model.evaluate(test_X,test_y)
print(f'Test Accuracy: {accuracy:.2f}')
print(f'Test Loss: {loss:.2f}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9991 - loss: 0.0060     
Test Accuracy: 1.00
Test Loss: 0.02


In [ ]:
model.save(r"my_model.keras")